In [44]:
from typing import Optional
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel

In [16]:
builder = StateGraph(dict)
builder

In [18]:
def addition(state):
    print(f"init_state: {state}")
    return {"x": state["x"] + 1}

def subtraction(state):
    print(f"addition_state: {state}")
    return {"x": state["x"] - 2}

In [20]:
builder.add_node("addition", addition)
builder.add_node("subtraction", subtraction)

builder.add_edge(START, "addition")
builder.add_edge("addition", "subtraction")
builder.add_edge("subtraction", END)

In [24]:
graph = builder.compile()

In [26]:
initial_state = {"x": 10}

graph.invoke(initial_state)

init_state: {'x': 10}
addition_state: {'x': 11}


{'x': 9}

In [28]:
state = {"x": 1, "y": "hello"}
state["x"] = "abc"
state

{'x': 'abc', 'y': 'hello'}

In [34]:
class MyState(BaseModel):
    x: int
    y: str = "default"

state = MyState(x="123")
print(state.x)
print(state.y)

123
default


In [42]:
class CalcState(BaseModel):
    x: int

def addition(state: CalcState) -> CalcState:
    print(f"[addition] initial state: {state}")
    return CalcState(x=state.x + 1)

def subtraction(state: CalcState) -> CalcState:
    print(f"[subtraction] received state: {state}")
    return CalcState(x=state.x - 2)

builder = StateGraph(CalcState)

builder.add_node("addition", addition)
builder.add_node("subtraction", subtraction)

builder.add_edge(START, "addition")
builder.add_edge("addition", "subtraction")
builder.add_edge("subtraction", END)

graph = builder.compile()

initial_state = {"x": 10}
graph.invoke(initial_state)

[addition] initial state: x=10
[subtraction] received state: x=11


{'x': 9}

In [46]:
class MyState(BaseModel):
    x: int
    result: Optional[str] = None

def check_x(state: MyState) -> MyState:
    print(f"[check_x] Received state: {state}")
    return state

def is_even(state: MyState) -> bool:
    return state.x % 2 == 0

def handle_even(state: MyState) -> MyState:
    print("[handle_even] x is even number")
    return MyState(x=state.x, result="even")

def handle_odd(state: MyState) -> MyState:
    print("[handle_odd] x is odd number")
    return MyState(x=state.x, result="odd")

builder = StateGraph(MyState)

builder.add_node("check_x", check_x)
builder.add_node("handle_even", handle_even)
builder.add_node("handle_odd", handle_odd)

builder.add_conditional_edges("check_x", is_even, {
    True: "handle_even",
    False: "handle_odd"
})

builder.add_edge(START, "check_x")
builder.add_edge("handle_even", END)
builder.add_edge("handle_odd", END)

graph = builder.compile()
graph.invoke(MyState(x=4))

[check_x] Received state: x=4 result=None
[handle_even] x is even number


{'x': 4, 'result': 'even'}

In [48]:
class LoopState(BaseModel):
    x: int

def increment(state: LoopState) -> LoopState:
    print(f"[increment] x = {state.x}")
    return LoopState(x=state.x + 1)

def is_done(state: LoopState) -> bool:
    return state.x > 10

builder = StateGraph(LoopState)
builder.add_node("increment", increment)

builder.add_conditional_edges("increment", is_done, {
    True: END,
    False: "increment"
})

builder.add_edge(START, "increment")
graph = builder.compile()
final_state = graph.invoke(LoopState(x=6))
print(f"[Final Result] -> x = {final_state['x']}")

[increment] x = 6
[increment] x = 7
[increment] x = 8
[increment] x = 9
[increment] x = 10
[Final Result] -> x = 11
